# Como saber si mi caja negra esta bien entrenada?

Haciendo un repaso:

1. Obtuvimos datos utiles para resolver nuestro problema y los convertimos en una tabla
2. Separamos nuestra tabla en dos: una tabla de **Entrenamiento** y una de **Prueba**
3. Utilizamos la tabla de entrenamiento para Entrenar una caja negra que me permite hacer predicciones

**Ahora utilizaremos la tabla de Prueba** para saber si nuestra caja negra esta bien entrenada aprovechando que en esta tabla de Prueba tenemos **datos que la caja negra nunca ha visto** y de los cuales **sabemos cuales deberian ser las predicciones correctas**.

![Error de prediccion de caja negra](img/error_prediccion_caja_negra.png)

Como dijimos ya hay metricas definidas y bien estudiadas para poder medir que tan bien entrenada esta una caja negra. Utilizaremos distintas medidas segun sean de **Regresion** o de **Clasificacion** 

Ahora solo veremos una para cada metodo y mas adelante en esta unidad veremos mas.

**Para regresion:**
- **Error cuadratico medio**: es la suma de las diferencias elevadas al cuadrado. Las diferencias se refieren a la resta entre cada valor predecido y el real. Cuanto menor sea este error mejor entrenada estara nuestra caja negra de regresion.

**Para clasificacion:**
- **Porcentaje de aciertos (Accuracy)**: cuantas predicciones hemos acertado dividido el total de predicciones. Cuanto mayor sea la cantidad de aciertos mejor entrenada estara nuestra caja negra de clasificacion.

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_verdadero, y_predecido)

## Sesgo y Varianza (Bias y Variance)

**Varianza Alta:**
- Error de Entrenamiento:  1%
- Error de Prueba: 10%

**Sesgo Alto:**
- Error de Entrenamiento: 15%
- Error de Prueba: 16%

**Sesgo y Varianza Altos:**
- Error de Entrenamiento: 15%
- Error de Prueba: 30%

**Sesgo y Varianza Bajos (lo que buscamos):**
- Error de Entrenamiento:  0.5%
- Error de Prueba:  1.0%

## Otras metricas

**Para clasificacion:**
    
Cuando realizamos una clasificacion intervienen varios factores:
1. Puede haber muchos mas datos de una clase que de otra y nos llevara a un accuracy alto pero que no es util
2. La mayoria de los algoritmos de clasificacion nos da la probabilidad de que un dato sea de una clase u otra por lo que deberemos elegir un limite a partir del cual decimos que es de una clase u otra.
    
- Accuracy.
- Precision: TP / (TP + FP)
- Recall: TP / (TP + FN)
- F1 Score: 2 / (1/P + 1/R)
- ROC curve: the precision and recall measures for different sigmoid thresholds
- Area Under de Curve: is the measure of the ability of a classifier to distinguish between classes and is used as a summary of the ROC curve

**Para regresion:**

Ademas del error cuadratico medio otra metrica muy utilizada es R2 que permite obtener un valor mas ameno ya que varia entre 0 y 1.

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_verdadero, y_predecido)